In [9]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
#sys.path.append(os.path.abspath("..")) 
from src.training.train import train_autoencoder, train_cellfate

# Load Data

In [ ]:
# Loading the first time point of the time-lapse microscopy data
x_train = np.load('../data/images/train_images_augmented.npy')[:,0,:,:]
y_train = np.load('../data/labels/train_labels_augmented.npy')
x_test = np.load('../data/images/test_images.npy')[:,0,:,:]
y_test = np.load('../data/labels/test_labels.npy')

# Print the shape of the data
print("x_train.shape: ", x_train.shape, "y_train.shape: ", y_train.shape, "x_test.shape: ", x_test.shape, "y_test.shape: ", y_test.shape)

# Train AI4CellFate

Stage 1: Train Adversarial Autoencoder for a low number of epochs, to wait for the reconstruction losses to converge before we do any latent space engineering

In [ ]:
config_autoencoder = {
        'batch_size': 30,
        'epochs': 15,
        'learning_rate': 0.001,
        'seed': 42,
        'latent_dim': 2,
        'GaussianNoise_std': 0.003,
        'lambda_recon': 5,
        'lambda_adv': 1,
    }

lambda_autoencoder_results = train_autoencoder(config_autoencoder, x_train)
encoder = lambda_autoencoder_results['encoder']
decoder = lambda_autoencoder_results['decoder']
discriminator = lambda_autoencoder_results['discriminator']

Stage 2: Train full AI4CellFate model (using the trained AAE from stage 1), where we add contrastive and covariance losses to enforce the latent features to be optimised for the classification task and independent from each other (for interpretation)

In [ ]:
config_ai4cellfate = {
        'batch_size': 30,
        'epochs': 100,
        'learning_rate': 0.001,
        'seed': 42,
        'latent_dim': 2,
        'GaussianNoise_std': 0.003,
        'lambda_recon': 6,
        'lambda_adv': 4,
        'lambda_cov': 0.0001,
        'lambda_contra': 8,
    }
 
lambda_ae_cov_results = train_cellfate(config_ai4cellfate, encoder, decoder, discriminator, x_train, y_train, x_test, y_test) 
encoder = lambda_ae_cov_results['encoder']
decoder = lambda_ae_cov_results['decoder']
discriminator = lambda_ae_cov_results['discriminator']

# Visualise Latent Space

# Classify from Engineered Latent Space

In [ ]:
# show pretty conf matrix

# Visual Interpretation 